In [1]:
# Download repos with rigt branch name
%cd /content/
!git clone https://github.com/zldrobit/yolov5.git
%cd yolov5
!git checkout tf-android

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 3497 (delta 0), reused 1 (delta 0), pack-reused 3493
Receiving objects: 100% (3497/3497), 8.58 MiB | 30.20 MiB/s, done.
Resolving deltas: 100% (2234/2234), done.
/content/yolov5
Branch 'tf-android' set up to track remote branch 'tf-android' from 'origin'.
Switched to a new branch 'tf-android'


In [2]:
# Download weight files
!./weights/download_weights.sh

100% 14.5M/14.5M [00:00<00:00, 20.9MB/s]

100% 41.9M/41.9M [00:01<00:00, 32.1MB/s]

100% 91.6M/91.6M [00:02<00:00, 37.1MB/s]

100% 170M/170M [00:04<00:00, 40.7MB/s]



In [3]:
# Install requirements
%set_env PYTHONPATH=/content/yolov5
import os 
os.environ['PYTHONPATH'] += ":/content/yolov5" 
import sys 
sys.path.append("/content/yolov5") 

!pip install -r requirements.txt

!pip install tensorflow==2.3.1

env: PYTHONPATH=/content/yolov5
     |████████████████████████████████| 276kB 9.1MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=c07b754d717f62bb7b8255f2d8aed93a779c74b2f2c3e86dd08f2d9f8bbdae06
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 320.4MB 47kB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [4]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [5]:
# Representative dataset:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.9.0/train_samples_coco.tar.gz
!tar -xvf train_samples_coco.tar.gz
!rm train_samples_coco.tar.gz

train_samples/
train_samples/COCO_train2014_000000151669.jpg
train_samples/COCO_train2014_000000476125.jpg
train_samples/COCO_train2014_000000125586.jpg
train_samples/COCO_train2014_000000116003.jpg
train_samples/COCO_train2014_000000123579.jpg
train_samples/COCO_train2014_000000308302.jpg
train_samples/COCO_train2014_000000292620.jpg
train_samples/COCO_train2014_000000314852.jpg
train_samples/COCO_train2014_000000026924.jpg
train_samples/COCO_train2014_000000466191.jpg
train_samples/COCO_train2014_000000549236.jpg
train_samples/COCO_train2014_000000563597.jpg
train_samples/COCO_train2014_000000387976.jpg
train_samples/COCO_train2014_000000118104.jpg
train_samples/COCO_train2014_000000330391.jpg
train_samples/COCO_train2014_000000115218.jpg
train_samples/COCO_train2014_000000517882.jpg
train_samples/COCO_train2014_000000421619.jpg
train_samples/COCO_train2014_000000444312.jpg
train_samples/COCO_train2014_000000219385.jpg
train_samples/COCO_train2014_000000106003.jpg
train_samples/COCO_

Change the end of the /content/yolov5/models/tf.py file to the following:

In [ ]:
            # fp32 TFLite model export
            converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
            converter.allow_custom_ops = False
            converter.experimental_new_converter = True
            tflite_model = converter.convert()
            f = opt.weights.replace('.pt', '-fp32.tflite')  # filename
            open(f, "wb").write(tflite_model)
            print('\nTFLite export success, saved as %s' % f)

            # dr TFLite model export
            converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
            converter.allow_custom_ops = False
            converter.experimental_new_converter = True
            tflite_model = converter.convert()
            f = opt.weights.replace('.pt', '-dr.tflite')  # filename
            open(f, "wb").write(tflite_model)
            print('\nTFLite export success, saved as %s' % f)

            # fp16 TFLite model export
            converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.target_spec.supported_types = [tf.float16]
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
            converter.allow_custom_ops = False
            converter.experimental_new_converter = True
            tflite_model = converter.convert()
            f = opt.weights.replace('.pt', '-fp16.tflite')  # filename
            open(f, "wb").write(tflite_model)
            print('\nTFLite export success, saved as %s' % f)

            # int8 float fallback TFLite model export
            converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.representative_dataset = representative_dataset_gen
            converter.allow_custom_ops = False
            converter.experimental_new_converter = True
            tflite_model = converter.convert()
            f = opt.weights.replace('.pt', '-fallback_int8.tflite')  # filename
            open(f, "wb").write(tflite_model)
            print('\nTFLite (int8) export success, saved as %s' % f)

            # int8 TFLite model export
            converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            converter.representative_dataset = representative_dataset_gen
            converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
            converter.inference_input_type = tf.uint8  # or tf.int8
            converter.inference_output_type = tf.uint8  # or tf.int8
            converter.allow_custom_ops = False
            converter.experimental_new_converter = True
            tflite_model = converter.convert()
            f = opt.weights.replace('.pt', '-full_int8.tflite')  # filename
            open(f, "wb").write(tflite_model)
            print('\nTFLite (int8) export success, saved as %s' % f)

NameError: ignored

Furthermore, add the following before the main funciont: (IMAGE SIZE IS FIXED TO 320!)

In [ ]:
rep_ds = tf.data.Dataset.list_files("./train_samples/*.jpg")

def representative_dataset_gen():
    size = 320
    for image_path in rep_ds:
        img = tf.io.read_file(image_path)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        resized_img = tf.image.resize(img, (size, size))
        resized_img = resized_img[tf.newaxis, :]
        yield [resized_img]

### **After all are done, run the following scrip:**

In [6]:
!PYTHONPATH=. python3  models/tf.py --weight yolov5s.pt --cfg models/yolov5s.yaml --img 320 --no-tfl-detect

2020-12-01 16:05:36.723990: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(batch_size=1, cfg='models/yolov5s.yaml', img_size=[320, 320], iou_thres=0.5, ncalib=100, no_tfl_detect=True, score_thres=0.4, source='/dataset/coco/coco2017/train2017', tf_nms=False, tfl_int8=False, topk_all=100, topk_per_class=100, weights='yolov5s.pt')

Starting TensorFlow saved_model export with TensorFlow 2.3.1...
2020-12-01 16:05:39.952436: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-01 16:05:40.013793: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-01 16:05:40.013855: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ba05008613c9): /proc/driver/nvidia/version does not exist
2020-12-01 16:05:4

In [ ]:
#Install the latest edge TPU compiler
%%bash
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu-compiler

In [ ]:
#Compile the file
!edgetpu_compiler -s /content/yolov5/yolov5s-full_int8.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 117 ms.

Input model: /content/yolov5/yolov5s-full_int8.tflite
Input size: 7.58MiB
Output model: yolov5s-full_int8_edgetpu.tflite
Output size: 7.51MiB
On-chip memory used for caching model parameters: 0.00B
On-chip memory remaining for caching model parameters: 8.09MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 205
Operation log: yolov5s-full_int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 1
Number of operations that will run on CPU: 204

Operator                       Count      Status

TRANSPOSE                      3          O

In [ ]:
#Download models (fp16, int8)
from google.colab import files
import glob

%cd /content/yolov5/

for file in glob.glob('./*.tflite'):
   files.download(file)

/content/yolov5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!mv /content/yolov5/yolov5s.tflite /content/yolov5s_320.tflite

In [ ]:
!PYTHONPATH=. python3  models/tf.py --weight yolov5s.pt --cfg models/yolov5s.yaml --img 160 --no-tfl-detect

2020-12-01 09:42:53.125216: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(batch_size=1, cfg='models/yolov5s.yaml', img_size=[160, 160], iou_thres=0.5, ncalib=100, no_tfl_detect=True, score_thres=0.4, source='/dataset/coco/coco2017/train2017', tf_nms=False, tfl_int8=False, topk_all=100, topk_per_class=100, weights='yolov5s.pt')

Starting TensorFlow saved_model export with TensorFlow 2.3.1...
2020-12-01 09:42:55.307775: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-01 09:42:55.317381: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-01 09:42:55.317428: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (961ce2359049): /proc/driver/nvidia/version does not exist
2020-12-01 09:42:5

In [ ]:
!mv /content/yolov5/yolov5s.tflite /content/yolov5s_160.tflite

In [ ]:
!PYTHONPATH=. python3  models/tf.py --weight yolov5s.pt --cfg models/yolov5s.yaml --img 480 --no-tfl-detect

2020-12-01 09:44:36.334387: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(batch_size=1, cfg='models/yolov5s.yaml', img_size=[480, 480], iou_thres=0.5, ncalib=100, no_tfl_detect=True, score_thres=0.4, source='/dataset/coco/coco2017/train2017', tf_nms=False, tfl_int8=False, topk_all=100, topk_per_class=100, weights='yolov5s.pt')

Starting TensorFlow saved_model export with TensorFlow 2.3.1...
2020-12-01 09:44:38.713290: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-01 09:44:38.722871: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-01 09:44:38.722921: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (961ce2359049): /proc/driver/nvidia/version does not exist
2020-12-01 09:44:3

In [ ]:
!mv /content/yolov5/yolov5s.tflite /content/yolov5s_480.tflite

In [ ]:
!PYTHONPATH=. python3  models/tf.py --weight yolov5s.pt --cfg models/yolov5s.yaml --img 640 --no-tfl-detect

2020-12-01 09:46:20.370262: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(batch_size=1, cfg='models/yolov5s.yaml', img_size=[640, 640], iou_thres=0.5, ncalib=100, no_tfl_detect=True, score_thres=0.4, source='/dataset/coco/coco2017/train2017', tf_nms=False, tfl_int8=False, topk_all=100, topk_per_class=100, weights='yolov5s.pt')

Starting TensorFlow saved_model export with TensorFlow 2.3.1...
2020-12-01 09:46:22.901942: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-01 09:46:22.911468: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-01 09:46:22.911513: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (961ce2359049): /proc/driver/nvidia/version does not exist
2020-12-01 09:46:2

In [ ]:
!mv /content/yolov5/yolov5s.tflite /content/yolov5s_640.tflite

In [ ]:
!PYTHONPATH=. python3  models/tf.py --weight yolov5s.pt --cfg models/yolov5s.yaml --img 800 --no-tfl-detect

2020-12-01 09:48:03.835039: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Namespace(batch_size=1, cfg='models/yolov5s.yaml', img_size=[800, 800], iou_thres=0.5, ncalib=100, no_tfl_detect=True, score_thres=0.4, source='/dataset/coco/coco2017/train2017', tf_nms=False, tfl_int8=False, topk_all=100, topk_per_class=100, weights='yolov5s.pt')

Starting TensorFlow saved_model export with TensorFlow 2.3.1...
2020-12-01 09:48:06.744154: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-12-01 09:48:06.753739: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-12-01 09:48:06.753783: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (961ce2359049): /proc/driver/nvidia/version does not exist
2020-12-01 09:48:0

In [ ]:
!mv /content/yolov5/yolov5s.tflite /content/yolov5s_800.tflite

In [ ]:
#Download different sized models
from google.colab import files
import glob
%cd /content/
for file in glob.glob('./*.tflite'):
   files.download(file)

/content


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
import numpy as np
import cv2
import requests
from io import BytesIO
from PIL import Image
from google.colab.patches import cv2_imshow
import time
import json
import tensorflow as tf


import os
import glob

In [8]:
# Pretrained classes in the model
%cd /content/
!wget https://gist.githubusercontent.com/aallan/fbdf008cffd1e08a619ad11a02b74fa8/raw/4183a4fd800c2d2d6211fd5129bf88d709072564/coco_labels.txt

def id_class_name(class_id):
    with open('coco_labels.txt', 'r') as f:
        for line in f:
            key, val = line.split()[0], ' '.join(line.split()[1:])
            if class_id == int(key):
                return val

/content
--2020-12-01 16:08:33--  https://gist.githubusercontent.com/aallan/fbdf008cffd1e08a619ad11a02b74fa8/raw/4183a4fd800c2d2d6211fd5129bf88d709072564/coco_labels.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 851 [text/plain]
Saving to: ‘coco_labels.txt’

coco_labels.txt     100%[===================>]     851  --.-KB/s    in 0s      

2020-12-01 16:08:34 (48.0 MB/s) - ‘coco_labels.txt’ saved [851/851]



In [9]:
# Load, save and display images:
urls = [
        "https://thehappypuppysite.com/wp-content/uploads/2017/10/Cute-Dog-Names-HP-long.jpg",
        "https://img.webmd.com/dtmcms/live/webmd/consumer_assets/site_images/articles/health_tools/cat_people_vs_dog_people_slideshow/getty_rm_photo_of_woman_running_with_dog.jpg",
        "https://media.glassdoor.com/l/04/4e/b7/18/celebrating-another-birthday-with-pizza-and-cake.jpg"
        ]

images = []
!mkdir test_images 

for url in urls:
    resp = requests.get(url, allow_redirects=True)
    filename = url.split('/')[-1]
    open('/content/test_images/' + filename, 'wb').write(resp.content)

    image = Image.open(BytesIO(resp.content))
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    images.append(image)

In [12]:
img_res = []
for image in images:
    img = image.copy()

    # Read the graph and run with tensorflow:
    # file = '/content/' + model_name + '/frozen_inference_graph.pb'
    file = '/content/yolov5/yolov5s.pb'
    with tf.io.gfile.GFile(file, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.compat.v1.Session() as sess:
        # Restore session
        sess.graph.as_default()
        tf.import_graph_def(graph_def, name='')

        # Read and preprocess an image.
        imH, imW, _ = img.shape
        inp = cv2.resize(img, (320, 320))
        inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

        # Run the model
        start_time = time.time()
        output_names = {
        'output_boxes': 'filtered_detections/map/TensorArrayStack/TensorArrayGatherV3:0',
        'output_scores': 'filtered_detections/map/TensorArrayStack_1/TensorArrayGatherV3:0',
        'output_labels': 'filtered_detections/map/TensorArrayStack_2/TensorArrayGatherV3:0'
        }
        output_boxes = sess.graph.get_tensor_by_name(output_names["output_boxes"])
        output_scores = sess.graph.get_tensor_by_name(output_names['output_scores'])
        output_labels = sess.graph.get_tensor_by_name(output_names['output_labels'])

    # graph = tf.Graph()
    # graph.as_default()
    # sess = tf.Session()
    # graph = get_frozen_graph(model_path)
    # tf.import_graph_def(graph, name='')



        out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                        sess.graph.get_tensor_by_name('detection_scores:0'),
                        sess.graph.get_tensor_by_name('detection_boxes:0'),
                        sess.graph.get_tensor_by_name('detection_classes:0')],
                      feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
        end_time = time.time()
        print("Inference time: ", end_time-start_time)

        # Visualize detected bounding boxes.
        num_detections = int(out[0][0])
        for i in range(num_detections):
            classId = int(out[3][0][i])
            class_name=id_class_name(classId)
            score = float(out[1][0][i])
            bbox = [float(v) for v in out[2][0][i]]
            if score > 0.45:
                ymin = int(max(1, bbox[0] * imH))
                xmin = int(max(1, bbox[1] * imW))
                ymax = int(min(imH, bbox[2] * imH))
                xmax = int(min(imW, bbox[3] * imW))
                
                cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

                # Draw label
                label = '%s: %d%%' % (class_name, int(score*100)) # Example: 'person: 72%'
                labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
                label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
                cv2.rectangle(img, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
                cv2.putText(img, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text
        #Visualize result(s)
        cv2_imshow(img)

KeyError: ignored